In [ ]:
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from time import sleep

# Set up the Chrome driver
options = webdriver.ChromeOptions()
#options.add_argument('--headless')
driver = webdriver.Chrome('./chromedriver', options=options)
wait = WebDriverWait(driver, 10)


url = 'https://www.giustizia-amministrativa.it/web/guest/dcsnprr?p_p_id=GaSearch_INSTANCE_2NDgCF3zWBwk&p_p_state=normal&p_p_mode=view&_GaSearch_INSTANCE_2NDgCF3zWBwk_javax.portlet.action=searchProvvedimenti&p_auth=mtYPa33V&p_p_lifecycle=0'
driver.get(url)

text_name = '_GaSearch_INSTANCE_2NDgCF3zWBwk_searchtextProvvedimenti'
input_field = driver.find_element("name",text_name)

input_field.send_keys("sentenza")




dropdown_element = driver.find_element("name", '_GaSearch_INSTANCE_2NDgCF3zWBwk_pageResultsProvvedimenti')
dropdown = Select(dropdown_element)
dropdown.select_by_value("60")

input_field.send_keys(Keys.ENTER)


#wait.until(EC.visibility_of_element_located((By.XPATH, '//div[@class="col-sm-9 risultati--ricerca"]')))
wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "ricerca--item")))
#sleep(10)
content = driver.page_source
nextPage(driver)
#driver.quit()

In [29]:
def getPageElement(page_content):
    page = BeautifulSoup(page_content, 'html.parser')
    footer = page.find_all('article')
    text = footer[60].find('div',{'class':'col-sm-12'}).text
    text = text.split('Risultati da ')[1].split(' ')
    print(text[2],text[4])
    return text[2],text[4]

getPageElement(content)

60 1338992


('60', '1338992')

In [27]:
driver.quit()

In [79]:
from bs4 import BeautifulSoup

content = driver.page_source

# Get the page source and parse it with BeautifulSoup
#content = driver.page_source
soup = BeautifulSoup(content, 'html.parser')

articles = soup.find_all('article')
# Extract the data you want
print(len(articles))


61


In [86]:
#print(articles[0])
prefix = 'https://www.giustizia-amministrativa.it'

PDF = []
ID = []
for article in articles:
    linkToPDF = article.find('a', {'class': 'visited-provvedimenti', 'title': 'Versione highlighted'})
    if(linkToPDF == None):
        continue
    PDFurl = prefix + linkToPDF['href']
    ID.append(linkToPDF.text)
    PDF.append(PDFurl)

print(len(PDF))
print(len(ID))
print(ID)

print(getPDFs([content])[1][0])

60
60
['202306351 (ROMA, SEZIONE 2B) html', '202306345 (ROMA, SEZIONE 1B) html', '202306344 (ROMA, SEZIONE 5B) html', '202306342 (ROMA, SEZIONE 2T) html', '202306341 (ROMA, SEZIONE 2T) html', '202306340 (ROMA, SEZIONE 2T) html', '202306339 (ROMA, SEZIONE 2T) html', '202306338 (ROMA, SEZIONE 2T) html', '202306337 (ROMA, SEZIONE 2T) html', '202306336 (ROMA, SEZIONE 2T) html', '202306334 (ROMA, SEZIONE 2T) html', '202306333 (ROMA, SEZIONE 2T) html', '202306332 (ROMA, SEZIONE 2T) html', '202306331 (ROMA, SEZIONE 3T) html', '202306330 (ROMA, SEZIONE 2S) html', '202306329 (ROMA, SEZIONE 2) html', '202306328 (ROMA, SEZIONE 2S) html', '202306327 (ROMA, SEZIONE 2S) html', '202306325 (ROMA, SEZIONE 2) html', '202306324 (ROMA, SEZIONE 2) html', '202306323 (ROMA, SEZIONE 2) html', '202306322 (ROMA, SEZIONE 2) html', '202306321 (ROMA, SEZIONE 2S) html', '202306319 (ROMA, SEZIONE 2S) html', '202306318 (ROMA, SEZIONE 1Q) html', '202306315 (ROMA, SEZIONE 2) html', '202306314 (ROMA, SEZIONE 2) html', '

In [96]:
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from time import sleep

def scrapeData(earlyStopping = False):
    pages = []
    # Set up the Chrome driver
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    driver = webdriver.Chrome('./chromedriver', options=options)
    wait = WebDriverWait(driver, 10)
    
    #Set the url
    url = 'https://www.giustizia-amministrativa.it/web/guest/dcsnprr?p_p_id=GaSearch_INSTANCE_2NDgCF3zWBwk&p_p_state=normal&p_p_mode=view&_GaSearch_INSTANCE_2NDgCF3zWBwk_javax.portlet.action=searchProvvedimenti&p_auth=mtYPa33V&p_p_lifecycle=0'
    driver.get(url)

    #Set the query correctly
    text_name = '_GaSearch_INSTANCE_2NDgCF3zWBwk_searchtextProvvedimenti'
    input_field = driver.find_element("name",text_name)

    input_field.send_keys("sentenza")

    dropdown_element = driver.find_element("name", '_GaSearch_INSTANCE_2NDgCF3zWBwk_pageResultsProvvedimenti')
    dropdown = Select(dropdown_element)
    dropdown.select_by_value("60")

    #Press enter to start the search
    input_field.send_keys(Keys.ENTER)


    #Extract the page
    page = extractPage(wait,driver)
    pages.append(page)
    current,total = getPageElement(page)

    #Extract the other pages 
    if(earlyStopping != False):
        total = earlyStopping
    while(int(current) < int(total)):
        nextPage(driver)
        page = extractPage(wait,driver)
        pages.append(page)
        current,total = getPageElement(page)
        if(earlyStopping != False):
            total = earlyStopping


    driver.quit()

    return pages

def nextPage(dr):
    nextPageButton = dr.find_element(By.LINK_TEXT, 'Successivo')
    nextPageButton.click()
    #print(nextPageButton)


def extractPage(w,d):
    #wait.until(EC.visibility_of_element_located((By.XPATH, '//div[@class="col-sm-9 risultati--ricerca"]')))
    w.until(EC.visibility_of_element_located((By.CLASS_NAME, "ricerca--item")))
    #sleep(10)
    content = d.page_source
    return content

In [87]:
def getPDFs(pages):
    PDF = []
    ID = []

    for page in pages:
        listOfItems = BeautifulSoup(page, 'html.parser')
        articles = soup.find_all('article')
        prefix = 'https://www.giustizia-amministrativa.it'
        
        for article in articles:
            linkToPDF = article.find('a', {'class': 'visited-provvedimenti', 'title': 'Versione highlighted'})
            if(linkToPDF == None):
                continue
            PDFurl = prefix + linkToPDF['href']
            ID.append(linkToPDF.text)
            PDF.append(PDFurl)

    print(f'Extracted {len(PDF)} PDFs and {len(ID)} descriptions')
    return PDF,ID

In [97]:

pagine = scrapeData(61)
print(f'Estratte {len(pagine)} pagine da 60 sentenze!')

PDFs,IDs = getPDFs(pagine)

/var/folders/l8/hdp9mfm55nq2xqyctskbfdvr0000gn/T/ipykernel_50816/1003509641.py:14: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver', options=options)


60 1338992
120 1338992
Estratte 2 pagine da 60 sentenze!
Extracted 120 PDFs and 120 descriptions


In [99]:
print(IDs)
print(PDFs[0])

['202306351 (ROMA, SEZIONE 2B) html', '202306345 (ROMA, SEZIONE 1B) html', '202306344 (ROMA, SEZIONE 5B) html', '202306342 (ROMA, SEZIONE 2T) html', '202306341 (ROMA, SEZIONE 2T) html', '202306340 (ROMA, SEZIONE 2T) html', '202306339 (ROMA, SEZIONE 2T) html', '202306338 (ROMA, SEZIONE 2T) html', '202306337 (ROMA, SEZIONE 2T) html', '202306336 (ROMA, SEZIONE 2T) html', '202306334 (ROMA, SEZIONE 2T) html', '202306333 (ROMA, SEZIONE 2T) html', '202306332 (ROMA, SEZIONE 2T) html', '202306331 (ROMA, SEZIONE 3T) html', '202306330 (ROMA, SEZIONE 2S) html', '202306329 (ROMA, SEZIONE 2) html', '202306328 (ROMA, SEZIONE 2S) html', '202306327 (ROMA, SEZIONE 2S) html', '202306325 (ROMA, SEZIONE 2) html', '202306324 (ROMA, SEZIONE 2) html', '202306323 (ROMA, SEZIONE 2) html', '202306322 (ROMA, SEZIONE 2) html', '202306321 (ROMA, SEZIONE 2S) html', '202306319 (ROMA, SEZIONE 2S) html', '202306318 (ROMA, SEZIONE 1Q) html', '202306315 (ROMA, SEZIONE 2) html', '202306314 (ROMA, SEZIONE 2) html', '202306